In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import sweetviz as sv
import missingno as msno
import time 
import sys
import os
import csv

import requests
import re

from sklearn.compose import make_column_selector, ColumnTransformer, make_column_transformer
# sklearn.compose: The sklearn.compose module is a submodule of the sklearn library for machine learning in Python. It provides functions for creating complex preprocessing and modeling pipelines.
from sklearn.preprocessing import OneHotEncoder,StandardScaler,PolynomialFeatures,RobustScaler
#sklearn.preprocessing: The sklearn.preprocessing module is a submodule of the sklearn library that provides functions for preprocessing data, such as scaling and normalizing features, imputing missing values, and encoding categorical variables.
from sklearn.linear_model import Ridge,LinearRegression,Lasso, ElasticNet
# sklearn.linear_model: The sklearn.linear_model module is a submodule of the sklearn library that provides functions for fitting linear models for regression and classification.
from sklearn.pipeline import make_pipeline
# sklearn.pipeline: The sklearn.pipeline module is a submodule of the sklearn library that provides functions for creating and working with pipelines of transformers and models.
from sklearn.model_selection import train_test_split,GridSearchCV,learning_curve, RandomizedSearchCV, cross_val_score, KFold
# sklearn.model_selection: The sklearn.model_selection module is a submodule of the sklearn library that provides functions for splitting data into training and test sets, evaluating models using cross-validation, and hyperparameter tuning.
from sklearn.dummy import DummyRegressor
# sklearn.dummy: The sklearn.dummy module is a submodule of the sklearn library that provides simple dummy models for regression and classification.


from sklearn.impute import SimpleImputer
import numpy as np

from sklearn.ensemble import RandomForestClassifier
import pickle


In [6]:
df=pd.read_csv("/home/apprenant/Documents/DATA/loan_project/SBAnational.csv")

df['ApprovalFY'] = df['ApprovalFY'].replace('A', '', regex=True).astype(int)

/tmp/ipykernel_38199/2682208335.py:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv("/home/apprenant/Documents/DATA/loan_project/SBAnational.csv")


In [7]:
# create a list of all numeric columns
columns_to_transform_to_int = ["DisbursementGross","BalanceGross","ChgOffPrinGr","GrAppv","SBA_Appv"]


for col in columns_to_transform_to_int:
    df[col] = df[col].str.replace("$", "")
    df[col] = df[col].str.replace(",", "")
    df[col] = df[col].astype(float)
    

df['Term'] = df['Term'].astype(int)

df['LowDoc'] = df['LowDoc'].replace({'0': 'N'})
df = df[df['LowDoc'].isin(['N','Y'])]
df['RevLineCr'] = df['RevLineCr'].replace({'0': 'N', 'T':'Y'})
df = df[df['RevLineCr'].isin(['N','Y'])]

dictionnaire =  {"11":"Agriculture, forestry, fishing and hunting","72":"Accommodation and food services","21":"Mining, quarrying, and oil and gas extraction","22":
"Utilities","23":"Construction","31":"Manufacturing","32":"Manufacturing","33":"Manufacturing","42":"Wholesale trade","44":"Retail trade","45":"Retail trade","48":" Transportation and warehousing", "49":"Transportation and warehousing", "51":"Information","52":"Finance and insurance", "53":"Real estate and rental and leasing","54":"Professional, scientific, and technical services","55":"Management of companies and enterprises","56":"Administrative and support and waste management and remediation services","61":"Educational services","62":"Health care and social assistance","71":"Arts, entertainment, and recreation","81":"Other services (except public administration)", "92": "Public administration", "0" : "Other"}

df['NAICS'] = df['NAICS'].astype(str).str[:2]

df["NAICS"] = df["NAICS"].map(dictionnaire)

cols_to_drop = ['LoanNr_ChkDgt', 'Name', 'City', 'Zip', 'Bank', 'BankState', 'ApprovalDate', 'RetainedJob','ChgOffDate', 'DisbursementDate', 'DisbursementGross', 'BalanceGross', 'SBA_Appv', 'ChgOffPrinGr']
df_cleaned = df.copy()
df_cleaned.drop(columns=cols_to_drop, inplace=True)

df_cleaned.dropna(subset=['MIS_Status'], inplace=True)


/tmp/ipykernel_38199/2495417656.py:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df[col] = df[col].str.replace("$", "")


In [8]:
y = df_cleaned.MIS_Status
X = df_cleaned.drop(columns=['MIS_Status'])

In [9]:
def make_pipeline_to_ML(X,y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, train_size=0.8, random_state=42)
    numerical_features = make_column_selector(dtype_include=np.number)
    categorical_features = make_column_selector(dtype_exclude= np.number)
    numerical_pipeline = make_pipeline(SimpleImputer(strategy='mean'),StandardScaler(with_mean=False))
    categorical_pipeline = make_pipeline(SimpleImputer(strategy='most_frequent'),OneHotEncoder(handle_unknown='ignore'))
    preprocessor = make_column_transformer((numerical_pipeline, numerical_features),
                                    (categorical_pipeline, categorical_features)
                                    )
    return preprocessor, X_train, X_test, y_train, y_test


# Create the pipeline
preprocessor, X_train, X_test, y_train, y_test = make_pipeline_to_ML(X,y)

# Logistic Regression
![Logistic](https://miro.medium.com/max/1400/0*q3wC98LKNGFSFwdG.gif)

In [10]:
from sklearn.linear_model import LogisticRegression

# Add LogisticRegression to the pipeline
pipeline = make_pipeline(preprocessor, LogisticRegression(max_iter=10000, class_weight='balanced', n_jobs=-1))
print("pipeline ... OK")
print("---"*20)
# Fit the pipeline to the training data
pipeline.fit(X_train, y_train)
print("---"*20)
print("Fit ... OK")
# Make predictions on the test data
y_pred = pipeline.predict(X_test)
print("---"*20)
print("Predict ... OK")
score = pipeline.score(X_test, y_test)
print('Accuracy:', score)

from sklearn.metrics import classification_report, confusion_matrix, f1_score

y_pred = pipeline.predict(X_test)
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

print("f1_score_average : Macro :",f1_score(y_test, y_pred, average='macro'))

pipeline ... OK
------------------------------------------------------------
------------------------------------------------------------
Fit ... OK
------------------------------------------------------------
Predict ... OK
Accuracy: 0.7151141513948709
              precision    recall  f1-score   support

      CHGOFF       0.36      0.82      0.50     31281
       P I F       0.95      0.69      0.80    146334

    accuracy                           0.72    177615
   macro avg       0.66      0.76      0.65    177615
weighted avg       0.84      0.72      0.75    177615

[[ 25701   5580]
 [ 45020 101314]]
f1_score_average : Macro : 0.6520556851690158


In [31]:
from sklearn.linear_model import LogisticRegression

# Add LogisticRegression to the pipeline
pipeline = make_pipeline(preprocessor, LogisticRegression(C=0.001, penalty='l2',solver='lbfgs',max_iter=10000, class_weight='balanced', n_jobs=-1, fit_intercept=False,tol=0.0001))

print("pipeline ... OK")
print("---"*20)
# Fit the pipeline to the training data
pipeline.fit(X_train, y_train)
print("---"*20)
print("Fit ... OK")
# Make predictions on the test data
y_pred = pipeline.predict(X_test)
print("---"*20)
print("Predict ... OK")
score = pipeline.score(X_test, y_test)
print('Accuracy:', score)

from sklearn.metrics import classification_report, confusion_matrix, f1_score

y_pred = pipeline.predict(X_test)
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

print("f1_score_average : Macro :",f1_score(y_test, y_pred, average='macro'))

pipeline ... OK
------------------------------------------------------------
------------------------------------------------------------
Fit ... OK
------------------------------------------------------------
Predict ... OK
Accuracy: 0.7131379669509895
              precision    recall  f1-score   support

      CHGOFF       0.36      0.83      0.50     31281
       P I F       0.95      0.69      0.80    146334

    accuracy                           0.71    177615
   macro avg       0.66      0.76      0.65    177615
weighted avg       0.85      0.71      0.75    177615

[[ 25922   5359]
 [ 45592 100742]]
f1_score_average : Macro : 0.6512527497712916


C : C est un coefficient de régularisation qui contrôle la force de la pénalité dans la régression logistique. Plus C est grand, plus la régularisation est faible, ce qui peut conduire à un modèle plus complexe et à un ajustement sur-adapté.

penalty : spécifie la forme de la pénalité pour la régularisation, 'l1' pour la pénalité L1 (régularisation Lasso) et 'l2' pour la pénalité L2 (régularisation Ridge).

solver : 'saga' est un algorithme de résolution utilisé pour la régression logistique.

max_iter : spécifie le nombre maximal d'itérations pour la convergence de l'algorithme.

fit_intercept : spécifie si le modèle doit inclure une constante (intercept) ou non.

tol : spécifie la tolérance pour la convergence de l'algorithme, c'est-à-dire l'erreur maximale admissible.

In [23]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline

# Define the hyperparameters to be tuned
hyperparameters = {'logisticregression__C': [0.1],
# 'logisticregression__C': [0.001, 0.01, 0.1, 1, 10, 100],
                #    'logisticregression__penalty': ['l1', 'l2'],
                #    'logisticregression__fit_intercept': [True, False]
                   }

# Create the pipeline with the LogisticRegression model
pipeline = make_pipeline(preprocessor, LogisticRegression(solver='saga',max_iter=10000,tol=0.0001,class_weight='balanced',penalty='l1',fit_intercept=False))

# Create a GridSearchCV object to perform the search
grid = GridSearchCV(pipeline, hyperparameters, cv=5, scoring='f1_macro',n_jobs =-1, verbose = 6)





In [24]:
# Fit the GridSearchCV object to the training data
grid.fit(X_train, y_train)


Fitting 5 folds for each of 1 candidates, totalling 5 fits


KeyboardInterrupt: 

In [ ]:
# Print the best hyperparameters and the best score
print("Best hyperparameters:", grid.best_params_)
print("Best Score:", grid.best_score_)

# Use the best hyperparameters to make predictions on the test data
y_pred = grid.predict(X_test)

# Print the classification report and confusion matrix
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))